In [36]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np
import random

In [37]:
class ShowerEnv(Env):
    def reset(seed):
        pass
    def __init__(self):
        # 3 actions
        self.action_space = Discrete(3)
        # temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # start temp
        self.state = 38 + random.randint(-3,3)
        # total time
        self.shower_length = 60
        
    def step(self, action):
        # do action
        # action can take values 0,1,2....
        self.state += action - 1
        # continue timer
        self.shower_length -= 1
        # reward process
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        # shower done or not
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        #apply temp noise
        self.state += random.randint(-1,1)
        # set placeholder
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        # implementing game visulization
        pass
    def reset(self):
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        return self.state
        
    

In [38]:
env = ShowerEnv()

In [39]:
env.observation_space.sample()

array([6.2401676], dtype=float32)

In [40]:
episodes = 10
for episode in range(1, episodes +1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-30
Episode:2 Score:-2
Episode:3 Score:-12
Episode:4 Score:-4
Episode:5 Score:-56
Episode:6 Score:-40
Episode:7 Score:-42
Episode:8 Score:-44
Episode:9 Score:-48
Episode:10 Score:-28


In [41]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
# from keras.models import Sequential, Model
# from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input
# from keras.optimizers import Adam

In [42]:
states = env.observation_space.shape
actions = env.action_space.n

In [43]:
actions,states

(3, (1,))

In [44]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24,activation="relu", input_shape=states))
    model.add(Dense(24,activation="relu"))
    model.add(Dense(actions, activation="linear"))
    return model

In [45]:
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Reshape, LSTM


In [46]:
def neural_network_model(input, actions):
    model = Sequential()
    # Define CNN model
    print(input)
    model.add(Conv2D(64, kernel_size=(5, 5), input_shape=input))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, kernel_size=(3, 3), input_shape=input))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
    model.add(Dropout(0.3))
    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Reshape((1, 256)))
    # Add some memory
    model.add(LSTM(256))
    model.add(Dense(actions, activation='softmax'))
    model.summary()
    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    return model

In [47]:
model = build_model(states,actions)

/home/yash7312/SOC/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 24)             │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 24)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 723 (2.82 KB)

 Trainable params: 723 (2.82 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# del model

In [50]:
# from tensorflow.keras.

In [51]:
# # Policy
# policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr="eps", value_max=1, value_min=0.2, value_test=.0,
#                                   nb_steps=1e6)

In [52]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

In [61]:
vec_env = make_vec_env("CartPole-v1", n_envs=4)
model_a2c = A2C("MlpPolicy", vec_env, verbose=1)

Using cuda device


In [63]:
model_a2c.learn(total_timesteps=2500)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 182      |
|    ep_rew_mean        | 182      |
| time/                 |          |
|    fps                | 1114     |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -0.46    |
|    explained_variance | 0.000101 |
|    learning_rate      | 0.0007   |
|    n_updates          | 1276     |
|    policy_loss        | 0.571    |
|    value_loss         | 1.56     |
------------------------------------


In [64]:
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_3" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (4, 4)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(4, 4), dtype=float32)
  • training=False
  • mask=None

In [ ]:
import tensorflow as tf
# from rl.agents import DQNAgent
import keras-rl

SyntaxError: invalid syntax (3654436219.py, line 3)

In [ ]:
agent = DqnAgent(
    model=model,
    memory=SequentialMemory(limit=50000, window_length=1),
    policy=BoltzmanNQPolicy(),
    nb_actions=actions,
    nb_steps_warmup=10,
    target_q_model_update=0.01
)

In [ ]:
agent.compile(Adam(learning_rate=0.001), metrics=["mae"])
agent.fit(env, nb_steps=100000, visulaize=False, verbose=1)

In [ ]:
results = agent.test(env,nb_episodes=10,visualize=True)

In [ ]:
import numpy as np
print(np.mean(results.history["episode_reward"]))
env.close()

In [ ]:
from tf_agents.agents.dqn import dqn_agent
from tf_agents.policies.boltzmann_policy import BoltzmannPolicy
from tf_agents.networks.sequential import Sequential
# from rl.memory import SequentialMemory

AttributeError: module 'keras._tf_keras.keras' has no attribute '__internal__'

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannPolicy()
    memory = Sequential(limit=50000, window_length=1)
    dqn = dqn_agent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

: 

In [ ]:
dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

: 

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

: 